In [ ]:
# GoogleNet
# 2014'te sınıflandırma'da 1.olmuştur
# Goggle araştırmacıları tarafından geliştirilmiş bir model.
# GoogleLeNet derler LeNet altyapısı kullanmış
# AlexNet'ten daha az parametreye sahip, daha başarılı model.
# 22 katmandan oluşuyor.
# Inceptıon modullleri : getirdiği yenilik, diğerlerinde peş peşe conv vs uyguluyorduk, burada aynı anda yan yana uyguluyoruz.
# yani farklı filtreleri aynı görüntüye uygulayıp tek bir çıktıda birleştiriyoruz.
# tam bağımlı katmanlar yok.
# 5 milyon params
# paralel filtrelerden çıkan feature maplerin birleştirilmesi için boyutlarının aynı olması lazım!.
# örn 3x3 filtre ve 5x5 filtrede , 5x5 olana paddingi fazla ekleyip daha da küçültürüz.
# 1x1 conv ne işe yarar : bir tane sayı demektir, bir sayıyı bütün görüntüyle çarparak yeni görüntü elde ederiz:
# bottleneck denen darboğazvar, filtre sayısını azaltmak istiyoruz. x*x yerine, 1x1 kullanarak daha az işlemle yapar. 1x1ler genelde veriyi küçültmek için kullanılır.
# temel problem ne? Karmaşıklık çok fazla 
# filtre birleştirirken toplam çıktı ne olucak? boyutlar aynı olduğu için, filtre adetlerini toplarız.
# hesaplama maliyeti yüksek (854m operations(işlem))
# her bir katmanda filtre arttığı için problemdir:
# bunun için bottleneck (1x1 conv) ekleniyor, feature derinliğini düşürmek için. (bknz: maliyet düştü : 353m operations (işlem))
# önce bottlencek ile daraltma sonra işlemlerde derinleştiririz.
# peki geçiçi çıktıların amacı nedir : çok derin olduğu için ara katmanlarda çıktı alıyoruz ve son katmandaki çıktı ile karşılaştırabiliyoruz.

# GoogleNet ne getirdi:
# Cnn modellerinin yalnızca peş peşe gelen conv katmanlardan oluşamayacağını, ınceptıon gibi birden fazla paralel yapıların kurulabileceğini
# bu yapıların performansı ve hesaplama derinliğini arttırabileceğini göstermiştir.


In [ ]:
import tensorflow as tf
from tensorflow import keras


from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers.core import Flatten,Dense,Dropout
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D ,Input,AveragePooling2D,GlobalAveragePooling2D
from tensorflow.keras.layers import Concatenate
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD

In [ ]:
# Bunun için veri alıp okuyoruz.

train_path='./drive/MyDrive/Deep Learning/chest_xray/train'
test_path='./drive/MyDrive/Deep Learning/chest_xray/test'

trainGen=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1      # validation verisi ayırdı.
)

valGen=ImageDataGenerator(rescale=1./255)
testGen=ImageDataGenerator(rescale=1./255)

# Veri okuma yöntemleri
# flow : görüntüleri okursunuz, listeye atarsanız, liste içerisinden flow(okuyabilirsiniz) atabilirsiniz 
# flow_from_directory : bir directory'den oku listeye atma, ihtiyacım olduğunda getir

trainData=trainGen.flow_from_directory(
    train_path,               # bu directory'den oku
    target_size=(224,224),    # hedef ne olsun, tüm görüntüler 28'e 28 olsun dedik, ne kadar büyük okursak o kadar kaliteli görüntü çıkar.
    class_mode= 'binary',     # okuma türünü belirliyoruz  -categorical dersek= 2D one-hut encoding yapılmış şekilde okur, -binary dersek= binary şekilde bir bir okur
    batch_size=32,
    color_mode='rgb',
    subset='training'         
    )

valData=trainGen.flow_from_directory(
    test_path,           
    target_size=(224,224),    
    class_mode= 'binary',           
    batch_size=32,
    color_mode='rgb',
    subset='validation'      # ayırdığımız validation datayı kullandık.
    )

testData=trainGen.flow_from_directory(
    test_path,           
    target_size=(224,224),    
    class_mode= 'binary',           
    batch_size=32,
    color_mode='rgb',
    )

Found 4695 images belonging to 2 classes.
Found 62 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
def inception(x, filtreler):
  #1x1
  yol1=Conv2D(filters=filtreler[0],kernel_size=1,strides=1,padding='same',activation='relu')(x)

  #1x1 -> 3x3
  yol2=Conv2D(filters=filtreler[1][0],kernel_size=1,strides=1,padding='same',activation='relu')(x)
  yol2=Conv2D(filters=filtreler[1][1],kernel_size=1,strides=1,padding='same',activation='relu')(yol2)

  #1x1 -> 5x5
  yol3=Conv2D(filters=filtreler[2][0],kernel_size=1,strides=1,padding='same',activation='relu')(x)
  yol3=Conv2D(filters=filtreler[2][1],kernel_size=1,strides=1,padding='same',activation='relu')(yol3)

  #3x3 max -> 1x1
  yol4=MaxPooling2D(pool_size=(3,3),strides=1,padding='same')(x)
  yol4=Conv2D(filters=filtreler[3],kernel_size=1,strides=1,padding='same',activation='relu')(yol4)

  return Concatenate(axis=-1)([yol1,yol2,yol3,yol4])

In [ ]:
# Çıkış katmanı
def auxilary(x,name=None):
  layer=AveragePooling2D(pool_size=(5,5),strides=3,padding='valid')(x)
  layer=Conv2D(128,kernel_size=1,padding='same',activation='relu')(layer)
  layer=Flatten()(layer)
  layer=Dense(256,activation='relu')(layer)
  layer=Dense(1,activation='sigmoid',name=name)(layer) # buraya 1 yazdıkta diğer prob. 1000 oluyor.

  return layer

In [ ]:

#bir input tanımladık.
input=Input(shape=(224,224,3))

#stage 1
x=Conv2D(64,kernel_size=(7,7),strides=(2,2),padding='same',activation='relu')(input)
x=MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)                     #alexnetin sunduğu overlapping
x=Conv2D(192,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
x=MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')

#stage 2
x=inception(x,[64,(96,128),(16,32),32]) #3a
x=inception(x,[128,(128,192),(32,96),64]) #3b
x=MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)

#stage 3
x=inception=(x,[192,(96,208),(16,48),64]) #4a
aux1=auxilary(x,name='aux1')
x=inception(x,[160,(112,224),(24,64),64]) #4b
x=inception(x,[128,(128,256),(24,64),64]) #4c
x=inception(x,[112,(144,288),(32,64),64]) #4d
aux2=auxilary(x,name='aux2')
x=inception(x,[256,(160,320),(32,128),128]) #4e
x=MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)

#stage 4
x=inception(x,[256,(160,320),(32,128),128]) #5a
x=inception(x,[384,(192,384),(48,128),128]) #5b
x=AveragePooling2D(pool_size=(7,7),strides=1,padding='valid')(x)
x=Flatten()(x)
out=Dense(1,activation='sigmoid',name='cikis')(x)

model=Model(inputs=input,outputs=[out,aux1,aux2])
model.summary()

TypeError: ignored

In [ ]:
model.compile(loss='binary_crossentropy',optimize='sgd',metrics=['accuracy'])
history=model.fit(trainData,validation_data=valData,epochs=10,batch_size=32)